In [11]:
import sys
import json
import warnings
import numpy as np
import pandas as pd
from numpy import genfromtxt
#from tabulate import tabulate
import numpy.linalg as linalg
import matplotlib.pyplot as plt
#from prettytable import PrettyTable
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

# Getting rid of annoying warnings. 
warnings.filterwarnings('ignore')

# Segun - 358_cause_recode
# Kuni - 113_cause_recode
# Gotham - 130_infact_cause_recode
# Lukia - 39_cause_recode

# Importing Professor Provided data. 
#medical_transcriptons_data = genfromtxt('./data-from-canvas/medicaltranscriptions.csv', delimiter=',', skip_header=1)

# Importing CDC data. Make sure you have the files in the right folder on your machine.
data_2005 = pd.read_csv( './mortality-data/2005_data.csv', na_values=['NA','?'])
#data_2006 = pd.read_csv( './mortality-data/2006_data.csv', na_values=['NA','?'])
#data_2007 = pd.read_csv( './mortality-data/2007_data.csv', na_values=['NA','?'])
#data_2008 = pd.read_csv( './mortality-data/2008_data.csv', na_values=['NA','?'])
#data_2009 = pd.read_csv( './mortality-data/2009_data.csv', na_values=['NA','?'])
#data_2010 = pd.read_csv( './mortality-data/2010_data.csv', na_values=['NA','?'])
#data_2011 = pd.read_csv( './mortality-data/2011_data.csv', na_values=['NA','?'])
#data_2012 = pd.read_csv( './mortality-data/2012_data.csv', na_values=['NA','?'])
#data_2013 = pd.read_csv( './mortality-data/2013_data.csv', na_values=['NA','?'])
#data_2014 = pd.read_csv( './mortality-data/2014_data.csv', na_values=['NA','?'])
#data_2015 = pd.read_csv( './mortality-data/2015_data.csv', na_values=['NA','?'])

# Importing ICD Codes
with open ('C:/GitHub/Mortality-Analysis/mortality-data/2005_codes.json') as json_file: 
    icd_codes_2005 = json.load(json_file)

In [ ]:
# Data into varibales 

### Defining a Method to find the most frequent cause of Death in a Data Set

In [12]:
def FindMostFrequentDeath(data, icd_codes): 
    most_frequent_death = str(int(data.mode()[0])).zfill(3)
    for code in icd_codes: 
        if code == most_frequent_death: 
            return icd_codes[code]

In [13]:
print(FindMostFrequentDeath(data_2005['130_infant_cause_recode'], icd_codes_2005))

None


In [ ]:
# Histogram of 113_cause_recode
hist1 = data_2005.plot( y=['113_cause_recode'], bins=135, alpha=0.5, figsize=(16,4), kind='hist')

In [ ]:
# Display Frequency
#https://note.nkmk.me/python-pandas-value-counts/ -- Japanese web site
#u = data_2005['113_cause_recode'].unique() : Unique values
#print(u)
#[ 28  84  70  46  27  86  43 110  23  89 123 133  62 125  44 111  22  24
# 100  68 126  78  15  52  25  29  73 109  95  63  32  69  67  21 102   3
# 33  20 118  59 116  94  39 108 122  56 121 128  51  55  10  48  74 114
#  18   6  40  45  31 129  16  35  77  34  36  41  85   9 120  57  30  60
#   5 135 101  26  71  65  88  50  75  90  99  96 115 132  66   7 119  38
#  98  87  81  80  92  91  49  83 107 104 103 130   1  42 134   2 106   8
#  17  11  13  14]
#print(len(u)) -- 112
pd.set_option('display.max_rows', 200)
frq = data_2005['113_cause_recode'].value_counts()
print(frq)

In [ ]:
# From above results, the top 5 of '113_cause_recode' in 2005 data are
# 1st:  63 : 228656 --- All other forms of chronic ischemic heart disease (I20,I25.1-I25.9)
# 2nd 111 : 217898 --- All other diseases (Residual) (D65-E07,E15-E34,E65-F99,G04-G12,G23-G25,G31-H93, K00-K22,K29-K31,K50-K66,K71-K72,K75-K76,K83-M99, N13.0-N13.5,N13.7-N13.9, N14,N15.0,N15.8-N15.9,N20-N23,N28-N39,N41-N64,N80-N98)
# 3rd 27 : 159405 --- Malignant neoplasms of trachea, bronchus and lung (C33-C34)
# 4th 59 : 151312 --- Acute myocardial infarction (I21-I22)
# 5th 70 : 143789 --- Cerebrovascular diseases (I60-I69)

In [ ]:
# Frequency of ALL data
dt_05 = data_2005['113_cause_recode']
dt_06 = data_2006['113_cause_recode']
dt_07 = data_2007['113_cause_recode']
dt_08 = data_2008['113_cause_recode']
dt_09 = data_2009['113_cause_recode']
dt_10 = data_2010['113_cause_recode']
dt_11 = data_2011['113_cause_recode']
dt_12 = data_2012['113_cause_recode']
dt_13 = data_2013['113_cause_recode']
dt_14 = data_2014['113_cause_recode']
dt_15 = data_2015['113_cause_recode']

# concatenate all data
dt_all = pd.concat([dt_05, dt_06, dt_07, dt_08, dt_09, dt_10, dt_11, dt_12, dt_13, dt_14, dt_15])

frq_all = dt_all.value_counts()
print(frq_all)

In [ ]:
# Histogram of  '113_cause_recode' in ALL data
hall= dt_all.plot( y=['113_cause_recode'], bins=135, alpha=0.5, figsize=(16,4), kind='hist')

### For 39_cause_recode column

In [ ]:
# A function to print the indexes for top 10 frequent values
# k is the number of values
from collections import Counter
def top_k(numbers, k=10):
    c = Counter(numbers)
    most_common = [key for key, val in c.most_common(k)]

    return most_common


In [ ]:
print(top_k(data_2005["39_cause_recode"]))